# Hashtags generator

Ferramenta que deverá ser capaz de pegar no conteúdo das noticias em texto e produzir hashtags.

In [2]:
import os
import json
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import string

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
##### Read the json files with de news

In [5]:
for file in os.listdir("../../../extractors/Snopes/extractions")[0:1]:
    with open("../../../extractors/Snopes/extractions/" + file) as f:
        data = json.load(f)
        f.close()


### Modelos

Verficar a utilizaçao de modelos já treinados para extrair keywords

#### Using pre-trained language models

In [8]:
from keybert import KeyBERT

### https://pypi.org/project/keybert/

In [10]:
original_text = data['postText']
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(original_text, keyphrase_ngram_range=(1, 1))
keywords

/home/maria/.local/lib/python3.8/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
9it [00:00, 10.04it/s]


[[('crowe', 0.3279),
  ('maximus', 0.3426),
  ('flying', 0.3439),
  ('gladiator', 0.4291),
  ('airplane', 0.4845)],
 [('endless', 0.2996),
  ('article', 0.3202),
  ('pages', 0.3332),
  ('ad', 0.412),
  ('slideshow', 0.5039)],
 [('biggest', 0.3259),
  ('movie', 0.3331),
  ('hide', 0.3378),
  ('movies', 0.4488),
  ('bloopers', 0.6852)],
 [('mistakes', 0.3608),
  ('actors', 0.3949),
  ('imperfections', 0.3968),
  ('movies', 0.5057),
  ('films', 0.5365)],
 [('misleading', 0.2373),
  ('gadot', 0.298),
  ('clickbait', 0.3363),
  ('airplane', 0.3797),
  ('gladiator', 0.3952)],
 [('movie', 0.202),
  ('sky', 0.3056),
  ('flying', 0.341),
  ('airplane', 0.4392),
  ('gladiator', 0.4851)],
 [('movie', 0.2619),
  ('thumbnail', 0.2726),
  ('fake', 0.3143),
  ('youtube', 0.3207),
  ('mistakes', 0.3351)],
 [('articles', 0.2499),
  ('ad', 0.3087),
  ('ads', 0.3662),
  ('clickbait', 0.4219),
  ('slideshow', 0.4429)],
 [('advertiser', 0.4064),
  ('ads', 0.4548),
  ('slideshow', 0.461),
  ('advertising', 

### KeyWords

In [7]:
def clean(sentence):
    
        stop = set(stopwords.words('english')) #define stop words
        exclude = set(string.punctuation) #define ponctuation for exclude
        lemma = WordNetLemmatizer() 
        
        stop_free = " ".join([i for i in sentence.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()) ##lematizar
        return normalized
    
    

In [8]:
import sentimentAnalysis

def sentiment_analysis(text, language):  
    
    status_code = 0 ## Valor inicial 
    
    while (status_code != 200):
        emotions, status_code = sentimentAnalysis.main(text,language)
    playload_emotions = json.loads(emotions)
    
    return playload_emotions
    

In [9]:
def hashtags_check_emotion(hashtaglist):
        ## call the module of sentiment analysis
        playload_emotions  = playload = sentiment_analysis(hashtaglist,'en')
   
        sortedByEmotion = {k: v for k, v in sorted(playload_emotions.items(), key=lambda item: (item[1]['emotions']['sadness'] + item[1]['emotions']['surprise'] + item[1]['emotions']['trust'] + item[1]['emotions']['anger'] + item[1]['emotions']['disgust'] + item[1]['emotions']['fear'] + item[1]['emotions']['joy']), reverse=True)}
       # print(sortedByEmotion)
        playload = [] ## add only the hashtags with some percentage of emotion
        
        
        for hashtag, emotion in sortedByEmotion.items():
            if (emotion['emotions']['sadness'] + emotion['emotions']['surprise'] + emotion['emotions']['trust'] + emotion['emotions']['anger'] + emotion['emotions']['disgust'] + emotion['emotions']['fear'] + emotion['emotions']['joy']) > 0:
                #print((emotion['emotions']['sadness'] + emotion['emotions']['surprise'] + emotion['emotions']['trust'] + emotion['emotions']['anger'] + emotion['emotions']['disgust'] + emotion['emotions']['fear'] + emotion['emotions']['joy']) > 0)
                playload.append(hashtag)
                
        return (playload)

In [12]:
#pip install keyphrase_vectorizers

In [13]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT


2022-04-28 11:17:14.572201: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-28 11:17:14.572243: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [19]:


def generateHashtags (text):
    topicsNun = 6
    passes = 50
    
   # text = ' '.join(data['postText']).split('\n')

    text_clean = []
    for sentence in text:
        d = clean(sentence).split()
        text_clean.append(d)

    text_clean = [clean(sentence).split() for sentence in text]   

    dictionary = corpora.Dictionary(text_clean)
    
    matrix = [dictionary.doc2bow(sentence) for sentence in text_clean]
    
    #Define the LDA model
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(matrix, num_topics=topicsNun, id2word = dictionary, passes=passes)
    #extract topics
    topic = ldamodel.print_topics(num_topics=topicsNun, num_words=5)
   

    
    
    ##Identify the keywords defined in the topics
    hashtags = []
    
    for t in topic:
        for h in t[1].split('+'):    
            hashtags.append(h[h.find('"')+1:h.rfind('"')])
    
    print("HashTags sugestion: ", list(set(hashtags)))
    
    
  #  kw_model = KeyBERT()
    
   # print("HashTags sugestion BERT: ", kw_model.extract_keywords(docs=text, keyphrase_ngram_range=(1,2)))
    
   
  #  print("HashTags sugestion BERT w/ vectorizer ", kw_model.extract_keywords(docs=text, vectorizer=KeyphraseCountVectorizer()))

    playload = hashtags_check_emotion (list(set(hashtags)))

    return (playload)

In [21]:
hashtags = generateHashtags(' '.join(data['postText']).split('\n'))
print(hashtags)
print(data['allegation'])
print(data['evaluation'])
print(data['url'])

HashTags sugestion:  ['twitter’s', 'account', 'rule', 'rumor', 'social', 'control', 'twitter', 'risk', 'musk', 'trump', '“due', 'medium', 'service', 'put']
['account', 'rule', 'rumor', 'trump', 'risk']
Twitter’s ban on former U.S. President Donald Trump was reversed in April 2022 by Elon Musk. 

false
https://www.snopes.com/fact-check/musk-reinstate-trump-on-twitter/


In [33]:
for file in os.listdir("extractors/Snopes/extractions")[:20]:
    with open("extractors/Snopes/extractions/" + file) as f:
        data = json.load(f)
        hashtags = generateHashtags(' '.join(data['postText']).split('\n'))
        print('alegation ',data['allegation'] ,'hashtags: ', hashtags)
        print('\n')
        f.close()

HashTags sugestion:  ['misleading', 'film', 'airplane', 'article', 'page', 'often', 'ad', 'movie', 'slideshow', '“gladiator”']
alegation  An airplane is visible in a scene in the movie “Gladiator,” which was set in ancient Rome.
 hashtags:  ['“gladiator”', 'misleading']


HashTags sugestion:  ['exchange', 'feature', 'several', 'twitter', 'swastika', 'nein…', 'emoji', 'pulling', 'branding', 'like', 'new', 'binance', 'symbol']
alegation  Cryptocurrency exchange company Binance briefly used a Twitter emoji that looked like a swastika.
 hashtags:  ['exchange']


HashTags sugestion:  ['twitter', 'rumor', 'put', 'service', 'medium', 'rule', 'musk', 'social', 'account', 'risk']
alegation  Twitter’s ban on former U.S. President Donald Trump was reversed in April 2022 by Elon Musk. 
 hashtags:  ['account', 'rule', 'rumor', 'risk']


HashTags sugestion:  ['account', 'remained', 'ban', 'posted', 'propaganda', 'intern', 'social', 'take', 'onion', 'twitter', 'banned']
alegation  The Twitter account